#### Step 0

0. defining location
1. defining peak power and technology type
2. define TRACKING_TYPE (fixed, single axis, two axes)
3. define RACKING_MODEL
4. define TILT (used both for fixed modules and single axis tracking)
5. define AZIMUTH (used both for fixed modules and single axis tracking)
6. define the surface type

based on such information, the user should be able to:

7. choose the module
8. choose the inverter

1) Goals & site

Location: Latitude/Longitude (map pin), auto-fill altitude (pvgis).
Target peak power (kWp) and Max area (m²) (optional).
Tech type (dropdown): TechChoice.
Tracking type (dropdown): TrackingType.
Mounting place (dropdown): RackingType
Azimuth (dial/compass) & Tilt (slider).
Surface type (dropdown → sets albedo): SurfaceType

In [1]:
from pvlib.pvsystem import retrieve_sam

from models.pv_custom.enums import RackingType, SurfaceType, TechChoice, TrackingType
from models.pv_custom.pv_plant import PVPlant

# input after selecting kwp, lat, lon, tech_choice
pv_plant = PVPlant(
    uri='',
    pod='IT000E00092',
    pod_uri='paidsfhgapghadg',
    kwp=6,
    maximum_area=0,
    latitude=41.9028,
    longitude=12.4964,
    angle=21.2,
    aspect=0,
    tech_choice=TechChoice.Mono_c_Si,
    tracking_type=TrackingType.fixed,
    mounting_place=RackingType.semi_integrated,
    surface_type=SurfaceType.concrete,
    percentage_loss=17,
    yearly_loss=0.5,
)

In [ ]:
from apis.pvlib_helper.geographical_helper import GeographicalHelper
from apis.pvlib_helper.pvsystems_helper import PVSystemHelper


pv_system_helper = PVSystemHelper()
cecmodules = retrieve_sam('cecmod')
inverters = retrieve_sam('cecinverter')
altitude = GeographicalHelper.get_elevation_opentopo(lat=pv_plant.latitude, lon=pv_plant.longitude)
hot_temperature, cold_temperature = GeographicalHelper.get_design_temperatures(
    lat=pv_plant.latitude, 
    lon=pv_plant.longitude, 
    altitude=altitude,
    surface_type=pv_plant.surface_type.name if pv_plant.surface_type is not None else None,
    racking_type=pv_plant.mounting_place.name,
    tilt_angle=pv_plant.angle
    )
filtered_inverters = pv_system_helper.filter_cec_inverters_by_kwp(pv_plant.kwp)
filtered_modules = pv_system_helper.filter_cec_modules_by_tech_choice(pv_plant.get_tech_choice_description)
filtered_modules = pv_system_helper.filter_cec_modules_by_inverters(filtered_modules, filtered_inverters, hot_temperature=hot_temperature, cold_temperature=cold_temperature)


In [4]:
filtered_inverters

,ABB__PVI_10_0_I_OUTD_x_US_208_y__208V_,ABB__PVI_10_0_I_OUTD_x_US_480_y_z__480V_,ABB__PVI_12_0_I_OUTD_x_US_480_y__480V_,ABB__PVI_5000_OUTD_S_US_Z__208V_,ABB__PVI_5000_OUTD_S_US_Z__240V_,ABB__PVI_5000_OUTD_S_US_Z__277V_,ABB__PVI_5000_OUTD_S_US_Z_A__208V_,ABB__PVI_5000_OUTD_S_US_Z_A__240V_,ABB__PVI_5000_OUTD_S_US_Z_A__277V_,ABB__PVI_5000_OUTD_US__208V_,...,Yaskawa_Solectria_Solar__PVI_7500_277__277V_,Yaskawa_Solectria_Solar__PVI_7600TL__208V_,Yaskawa_Solectria_Solar__PVI_7600TL__240V_,Yes!_Solar__ES5000__240V_,Yes!_Solar__ES5000P__240V_,Yes!_Solar__ES5400__240V_,Yes!_Solar__ES5400P__240V_,Zigor__SUNZET_5_TL_US__240V_,iPower__SHO_4_8__240V_,iPower__SHO_5_2__240V_
Vac,208,480,480,208,240,277,208,240,277,208,...,277,208,240,240,240,240,240,240,240,240
Pso,46.863808,67.790878,62.554672,30.859711,32.035774,38.20557,30.859711,32.035774,38.20557,30.859711,...,40.666569,35.01656,33.877434,30.92363,30.92363,29.625299,29.625299,32.086174,85.145699,62.486748
Paco,10000.0,10000.0,12000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,...,7460.0,6600.0,7600.0,4900.0,4900.0,5300.0,5300.0,5240.0,4800.0,5200.0
Pdco,10488.306641,10295.979492,12358.804688,5208.396484,5190.294434,5162.4375,5208.396484,5190.294434,5162.4375,5208.396484,...,7817.195313,6802.745605,7805.277832,5137.208008,5137.208008,5573.729004,5573.729004,5489.155762,4968.027832,5382.856934
Vdco,320.0,362.0,370.0,310.0,345.0,400.0,310.0,345.0,400.0,310.0,...,275.0,380.0,380.0,275.0,275.0,275.0,275.0,380.0,263.0,280.0
C0,-0.000003,-0.000001,-0.000001,-0.000004,-0.000004,-0.000003,-0.000004,-0.000004,-0.000003,-0.000004,...,-0.000004,-0.000004,-0.000003,-0.000006,-0.000006,-0.000006,-0.000006,-0.000004,-0.000006,-0.000005
C1,-0.000036,-0.000049,-0.000056,-0.000003,-0.00001,-0.000018,-0.000003,-0.00001,-0.000018,-0.000003,...,0.000007,-0.000026,-0.000032,0.000013,0.000013,0.000011,0.000011,-0.000074,0.000034,0.000044
C2,0.000305,-0.00052,-0.001437,0.00113,0.000445,0.000153,0.00113,0.000445,0.000153,0.00113,...,0.000928,-0.00039,-0.000495,0.001078,0.001078,0.000969,0.000969,-0.000083,0.000586,0.00126
C3,-0.002351,-0.003855,-0.007112,-0.000327,-0.000412,-0.000797,-0.000327,-0.000412,-0.000797,-0.000327,...,-0.000121,-0.000994,-0.001335,0.000068,0.000068,-0.000064,-0.000064,-0.001604,0.000195,0.000367
Pnt,0.1,0.4,0.4,1.5,1.5,1.5,1.5,1.5,1.5,1.5,...,2.238,1.98,2.28,1.47,1.47,1.59,1.59,1.572,1.44,1.56


In [ ]:
pv_plant.set_altitude(altitude)
pv_plant.set_module_names(filtered_modules.columns)
pv_plant.set_inverter_names(filtered_inverters.columns)

##### User chooses either a pv module OR an inverter

the app must, then, filter again the dropdown of the technology that the user has not picked yet.

In [4]:
inverters.T.head()

,Vac,Pso,Paco,Pdco,Vdco,C0,C1,C2,C3,Pnt,Vdcmax,Idcmax,Mppt_low,Mppt_high,CEC_Date,CEC_Type
ABB__MICRO_0_25_I_OUTD_US_208__208V_,208,2.089607,250.0,259.588593,40.0,-0.000041,-0.000091,0.000494,-0.013171,0.075,50.0,6.489715,30.0,50.0,NaN,Utility Interactive
ABB__MICRO_0_25_I_OUTD_US_240__240V_,240,2.240412,250.0,259.492065,40.0,-0.000039,-0.000132,0.002418,-0.014926,0.075,50.0,6.487302,30.0,50.0,NaN,Utility Interactive
ABB__MICRO_0_3_I_OUTD_US_208__208V_,208,1.846506,300.0,311.66925,40.0,-0.000033,-0.000192,0.000907,-0.031742,0.09,50.0,7.791731,30.0,50.0,NaN,Utility Interactive
ABB__MICRO_0_3_I_OUTD_US_240__240V_,240,1.950539,300.0,311.580872,40.0,-0.000034,-0.000256,0.002453,-0.028223,0.09,50.0,7.789522,30.0,50.0,NaN,Utility Interactive
ABB__MICRO_0_3HV_I_OUTD_US_208__208V_,208,1.769444,300.0,312.421082,45.0,-0.000045,-0.000196,0.001959,-0.023725,0.09,60.0,6.942691,30.0,60.0,NaN,Utility Interactive


#### Step 1

0. choosing the number of modules in series (n_series)
1. choosing the number of series in parallel (n_strings)
2. suggest an ideal number of inverters
3. choose number of inverters
4. output the actual peak power achieved

5. the app must tell the user when the configuration he is forcing is not feasible

#### Step 2

4. define MAX_ANGLE if it is single axis tracking
5. define BACKTRACK if it is single axis tracking (True vs False)
6. define GCR if it is single axis tracking (default 0.35)
7. define temperature parameters (rules for default parameters is defined in class RackingType)
8. define b0 parameter of the module for IAM coefficient (defulat 0.05) for the ashrae aoi model
0. define wiring material (WiringMaterial enum)
1. define wiring length
2. define wiring cross section
3. define AC_WIRING_FACTOR
4. define AVAILABILITY
5. define LID_FACTOR
6. define MODULE_QUALITY_FACTOR
7. define MISMATCH_FACTOR
8. extract SOILING_MONTHLY (Monthly soiling transmittance)

#### Step 3

2. fetch and preprocess (extract ghi, dhi, dni, airmass, precipitable_water) meteo data

Step 5

0. define pvlib.pvsystem.Array
1. define pvlib.pvsystem.PVSystem
2. define pvlib.modelchain.ModelChain
3. apply posterior loss model
4. multiply by the number of inverters

#### Step 6

0. Apply Markov Chain Montecarlo model to the 12 years of historical backcast of production
1. Save the ProductionMatrix
2. Simulate self consumption and grid injection